# Real data comparison

In [111]:
%pylab inline

import scipy.sparse
import pandas as pd
from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance

from regain import datasets; reload(datasets)
from regain import model_selection; reload(model_selection)
from regain.plot import plot_graph_with_latent_variables
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_)
from regain.utils import error_norm

Populating the interactive namespace from numpy and matplotlib


In [112]:
df = pd.read_excel("/home/fede/Desktop/inline-supplementary-material-2.xls", skiprows=1, index_col=0)
best_dims = pd.read_csv("/home/fede/Desktop/inline-supplementary-material-3.txt", delimiter="\t",
                            skiprows=[0,2], skipinitialspace=True)

In [113]:
df.columns = df.columns.str.lower()

In [114]:
best_dims['sum_abs_diff_stress'] = (np.abs(best_dims['cold_2'] - best_dims['cold_3']) + 
    np.abs(best_dims['heat_2'] - best_dims['heat_3']) +
    np.abs(best_dims['oxidative_2'] - best_dims['oxidative_3']))

In [115]:
dims = best_dims.sort_values("sum_abs_diff_stress", ascending=False).head(10)['condition_timepoint'].tolist()

In [116]:
for x in ['shikimic acid-3-phosphate', "methionine", '2-ketobutyric acid']:
    try:
        dims.remove(x)
    except:
        pass

In [117]:
df_reduced = df.loc[:,['time point', 'stress'] + dims].fillna(1)

In [118]:
df_reduced = df_reduced[df_reduced.stress.isin(['cold stress', 'heat stress', 'oxidative stress'])]

In [119]:
a3d = np.array(list(df.groupby("time point").apply(pd.DataFrame.as_matrix)))

df_group = df_reduced.groupby("time point")
# df_group = df_group.fillna(0)
min_size = int(np.array([group[dims].values.shape[0] for name, group in df_group if name < 12], dtype=float).min())
X = np.array([group[dims].values[:min_size] for name, group in df_group if name < 12], dtype=float)
# for name, group in df_group:
#     print group.values.astype(float)

In [121]:
data_list = np.log(X.copy())
data_grid = data_list.transpose(1,2,0)

In [ ]:
mdl = latent_time_graph_lasso_.LatentTimeGraphLasso(
    bypass_transpose=False, assume_centered=0, verbose=1, rtol=1e-5, tol=1e-5, phi='node',
    max_iter=250, rho=1./ np.sqrt(np.array(data_list).shape[1]))

# alpha=np.linspace(0.01, 0.2, 5)
# tau=np.linspace(0.01, 0.8, 5)
# beta=np.linspace(0.01, 0.8, 2)
# eta=np.linspace(0.01, 0.8, 2)
# params = dict(tau=tau, eta=eta, alpha=alpha, beta=beta)

# ltgl = GridSearchCV(mdl, params, cv=ShuffleSplit(3, test_size=.5), verbose=True).fit(data_grid)

domain = [
    {'name': 'tau', 'type': 'continuous', 'domain': (-3,np.log10(.25))},
    {'name': 'alpha', 'type': 'continuous', 'domain': (-2.,0.)},
    {'name': 'beta', 'type': 'continuous', 'domain': (-2.,0.)},
    {'name': 'eta', 'type': 'continuous', 'domain': (-2.,0.)},
]
ltgl = model_selection.BayesianOptimization(
    mdl, domain=domain, n_jobs=2, acquisition_weight=1, acquisition_jitter=.01,
    acquisition_type='EI', max_iter=200).fit(data_grid)

alpha = ltgl.best_params_['alpha']
tau = ltgl.best_params_['tau']
beta = ltgl.best_params_['beta']
eta = ltgl.best_params_['eta']

tic = time.time()
# ll = ltgl.best_estimator_.fit(data_grid)
ll = mdl.set_params(**ltgl.best_params_).set_params(max_iter=500).fit(data_grid)
tac = time.time()

obj: -17.0101, rnorm: 17.2336, snorm: 3.3798,eps_pri: 0.0007, eps_dual: 0.0005
obj: -2.5212, rnorm: 16.5017, snorm: 3.2363,eps_pri: 0.0007, eps_dual: 0.0005
obj: 9.6785, rnorm: 11.0357, snorm: 1.2279,eps_pri: 0.0006, eps_dual: 0.0006
obj: 21.7535, rnorm: 10.6261, snorm: 1.1685,eps_pri: 0.0006, eps_dual: 0.0006
obj: 24.7933, rnorm: 8.6555, snorm: 0.4710,eps_pri: 0.0006, eps_dual: 0.0006
obj: 35.5807, rnorm: 8.3628, snorm: 0.4495,eps_pri: 0.0006, eps_dual: 0.0006
obj: 42.5320, rnorm: 6.5542, snorm: 0.8306,eps_pri: 0.0006, eps_dual: 0.0006
obj: 51.9005, rnorm: 6.3580, snorm: 0.7955,eps_pri: 0.0006, eps_dual: 0.0006
obj: 54.3769, rnorm: 5.4633, snorm: 0.4310,eps_pri: 0.0006, eps_dual: 0.0006
obj: 62.8511, rnorm: 5.3147, snorm: 0.4139,eps_pri: 0.0006, eps_dual: 0.0006
obj: 69.7121, rnorm: 4.2474, snorm: 0.9179,eps_pri: 0.0005, eps_dual: 0.0007
obj: 77.0999, rnorm: 4.1351, snorm: 0.8864,eps_pri: 0.0005, eps_dual: 0.0006
obj: 80.0667, rnorm: 3.4662, snorm: 0.6532,eps_pri: 0.0005, eps_dual: 0.

obj: 98.6396, rnorm: 0.0198, snorm: 0.0043,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2348, rnorm: 0.0131, snorm: 0.0021,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6458, rnorm: 0.0191, snorm: 0.0032,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6523, rnorm: 0.0184, snorm: 0.0026,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2383, rnorm: 0.0121, snorm: 0.0045,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6598, rnorm: 0.0174, snorm: 0.0033,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2407, rnorm: 0.0116, snorm: 0.0019,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6674, rnorm: 0.0164, snorm: 0.0044,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6732, rnorm: 0.0158, snorm: 0.0024,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2436, rnorm: 0.0112, snorm: 0.0013,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6781, rnorm: 0.0152, snorm: 0.0019,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2448, rnorm: 0.0108, snorm: 0.0011,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.6823, rnorm: 0.0147, snorm: 0.0017,eps_pri: 0.0006, eps_dual: 0.0007

obj: 88.2526, rnorm: 0.0006, snorm: 0.0001,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7758, rnorm: 0.0019, snorm: 0.0003,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7759, rnorm: 0.0018, snorm: 0.0003,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2531, rnorm: 0.0006, snorm: 0.0001,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7763, rnorm: 0.0018, snorm: 0.0004,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2526, rnorm: 0.0006, snorm: 0.0001,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7761, rnorm: 0.0018, snorm: 0.0003,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2532, rnorm: 0.0006, snorm: 0.0001,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7761, rnorm: 0.0018, snorm: 0.0002,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2532, rnorm: 0.0006, snorm: 0.0001,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7771, rnorm: 0.0018, snorm: 0.0002,eps_pri: 0.0006, eps_dual: 0.0007
obj: 98.7761, rnorm: 0.0017, snorm: 0.0002,eps_pri: 0.0006, eps_dual: 0.0007
obj: 88.2526, rnorm: 0.0006, snorm: 0.0001,eps_pri: 0.0006, eps_dual: 0.0007

obj: 92.8612, rnorm: 0.0402, snorm: 0.0153,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.8807, rnorm: 0.0375, snorm: 0.0158,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.8982, rnorm: 0.0346, snorm: 0.0162,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9097, rnorm: 0.0327, snorm: 0.0138,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9167, rnorm: 0.0301, snorm: 0.0146,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9220, rnorm: 0.0269, snorm: 0.0132,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9235, rnorm: 0.0247, snorm: 0.0110,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9227, rnorm: 0.0236, snorm: 0.0075,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9237, rnorm: 0.0226, snorm: 0.0061,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9310, rnorm: 0.0209, snorm: 0.0091,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9371, rnorm: 0.0197, snorm: 0.0059,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9451, rnorm: 0.0184, snorm: 0.0069,eps_pri: 0.0006, eps_dual: 0.0007
obj: 92.9519, rnorm: 0.0179, snorm: 0.0037,eps_pri: 0.0006, eps_dual: 0.0007

obj: 93.2064, rnorm: 0.0023, snorm: 0.0016,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2074, rnorm: 0.0022, snorm: 0.0011,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2093, rnorm: 0.0022, snorm: 0.0008,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2110, rnorm: 0.0021, snorm: 0.0006,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2095, rnorm: 0.0021, snorm: 0.0005,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2102, rnorm: 0.0021, snorm: 0.0005,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2107, rnorm: 0.0021, snorm: 0.0005,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2112, rnorm: 0.0020, snorm: 0.0005,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2121, rnorm: 0.0020, snorm: 0.0005,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2121, rnorm: 0.0020, snorm: 0.0004,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2125, rnorm: 0.0020, snorm: 0.0004,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2141, rnorm: 0.0020, snorm: 0.0004,eps_pri: 0.0006, eps_dual: 0.0007
obj: 93.2136, rnorm: 0.0020, snorm: 0.0004,eps_pri: 0.0006, eps_dual: 0.0007

...

obj: 110.1269, rnorm: 0.0274, snorm: 0.0017,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.0452, rnorm: 0.0288, snorm: 0.0053,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1428, rnorm: 0.0250, snorm: 0.0046,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.0537, rnorm: 0.0262, snorm: 0.0060,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1549, rnorm: 0.0229, snorm: 0.0050,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1637, rnorm: 0.0210, snorm: 0.0050,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.0633, rnorm: 0.0240, snorm: 0.0060,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1700, rnorm: 0.0193, snorm: 0.0045,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.0736, rnorm: 0.0221, snorm: 0.0053,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1745, rnorm: 0.0178, snorm: 0.0036,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.0842, rnorm: 0.0204, snorm: 0.0042,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1782, rnorm: 0.0164, snorm: 0.0028,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.1817, rnorm: 0.0151, snorm: 0.0022,eps_pri: 0.0005, eps

obj: 90.8194, rnorm: 3.0124, snorm: 0.5408,eps_pri: 0.0005, eps_dual: 0.0007
obj: 102.1743, rnorm: 0.0011, snorm: 0.0001,eps_pri: 0.0005, eps_dual: 0.0008
obj: 96.3628, rnorm: 2.6748, snorm: 0.4637,eps_pri: 0.0005, eps_dual: 0.0007
obj: 102.1745, rnorm: 0.0010, snorm: 0.0001,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.1747, rnorm: 0.0010, snorm: 0.0001,eps_pri: 0.0005, eps_dual: 0.0008
obj: 100.6358, rnorm: 2.4289, snorm: 0.3226,eps_pri: 0.0005, eps_dual: 0.0007
obj: 102.1748, rnorm: 0.0009, snorm: 0.0001,eps_pri: 0.0005, eps_dual: 0.0008
obj: 104.1246, rnorm: 2.0196, snorm: 0.4808,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.1751, rnorm: 0.0008, snorm: 0.0005,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.4416, rnorm: 1.4235, snorm: 0.7661,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.1754, rnorm: 0.0007, snorm: 0.0004,eps_pri: 0.0005, eps_dual: 0.0008
obj: 107.2430, rnorm: 0.9050, snorm: 0.8505,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.1756, rnorm: 0.0007, snorm: 0.0003,eps_pri: 0.0005, eps_d

obj: 94.7289, rnorm: 2.7885, snorm: 0.3243,eps_pri: 0.0005, eps_dual: 0.0007
obj: 104.9665, rnorm: 2.4317, snorm: 0.2629,eps_pri: 0.0005, eps_dual: 0.0007
obj: 99.6237, rnorm: 2.4456, snorm: 0.3042,eps_pri: 0.0005, eps_dual: 0.0007
obj: 103.5557, rnorm: 1.8024, snorm: 0.6615,eps_pri: 0.0005, eps_dual: 0.0008
obj: 108.7737, rnorm: 1.9667, snorm: 0.5073,eps_pri: 0.0005, eps_dual: 0.0008
obj: 105.4646, rnorm: 1.0699, snorm: 0.8993,eps_pri: 0.0005, eps_dual: 0.0008
obj: 111.1722, rnorm: 1.4008, snorm: 0.7411,eps_pri: 0.0005, eps_dual: 0.0008
obj: 105.4945, rnorm: 0.6433, snorm: 0.8488,eps_pri: 0.0005, eps_dual: 0.0008
obj: 111.9600, rnorm: 0.9335, snorm: 0.7798,eps_pri: 0.0005, eps_dual: 0.0008
obj: 104.3897, rnorm: 0.6420, snorm: 0.5955,eps_pri: 0.0006, eps_dual: 0.0008
obj: 111.5832, rnorm: 0.7513, snorm: 0.6269,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.9300, rnorm: 0.6930, snorm: 0.2925,eps_pri: 0.0006, eps_dual: 0.0008
obj: 110.6524, rnorm: 0.6672, snorm: 0.4295,eps_pri: 0.0005, eps_d

obj: 110.6906, rnorm: 0.0055, snorm: 0.0007,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.6653, rnorm: 0.0065, snorm: 0.0014,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.6922, rnorm: 0.0051, snorm: 0.0006,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.6665, rnorm: 0.0061, snorm: 0.0015,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.6938, rnorm: 0.0047, snorm: 0.0006,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.6954, rnorm: 0.0044, snorm: 0.0005,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.6677, rnorm: 0.0057, snorm: 0.0010,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.6968, rnorm: 0.0041, snorm: 0.0005,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.6689, rnorm: 0.0053, snorm: 0.0013,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.6982, rnorm: 0.0038, snorm: 0.0005,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.6994, rnorm: 0.0035, snorm: 0.0004,eps_pri: 0.0005, eps_dual: 0.0008
obj: 102.6700, rnorm: 0.0048, snorm: 0.0015,eps_pri: 0.0005, eps_dual: 0.0008
obj: 110.7005, rnorm: 0.0033, snorm: 0.0004,eps_pri: 0.0005, eps

obj: 106.4631, rnorm: 0.0352, snorm: 0.0179,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.4705, rnorm: 0.0320, snorm: 0.0125,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.4737, rnorm: 0.0287, snorm: 0.0097,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.4775, rnorm: 0.0257, snorm: 0.0089,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.4851, rnorm: 0.0230, snorm: 0.0086,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.4971, rnorm: 0.0208, snorm: 0.0076,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5127, rnorm: 0.0184, snorm: 0.0065,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5299, rnorm: 0.0160, snorm: 0.0069,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5463, rnorm: 0.0143, snorm: 0.0057,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5597, rnorm: 0.0121, snorm: 0.0077,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5692, rnorm: 0.0101, snorm: 0.0071,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5753, rnorm: 0.0090, snorm: 0.0062,eps_pri: 0.0005, eps_dual: 0.0008
obj: 106.5778, rnorm: 0.0079, snorm: 0.0059,eps_pri: 0.0005, eps

In [ ]:
mdl.latent_

In [2]:
data = pd.read_csv("/home/fede/src/slipguru/regain/regain/data/finance.csv", header=0, index_col=0)

In [16]:
data.loc[:, ['A', 'AA']][10:12]

,A,AA
20090904,1.22,0.38
20090909,3.87,4.05


In [48]:
n = 30
data_list = np.array([data.loc[:,['AMZN', 'IBM', 'GOOG', 'MSFT','APOL']][i:i+n].values 
             for i in range(1, data.shape[0], n)][:-1])

In [60]:
[data.loc[:,['AMZN', 'IBM', 'GOOG', 'MSFT','APOL']][i:i+n] 
             for i in range(1, data.shape[0], n)][:-1]

[          AMZN   IBM  GOOG  MSFT  APOL
 20090824 -0.62 -0.52  0.72  0.91 -0.47
 20090825 -0.68 -0.73  0.25 -0.32  1.43
 20090826 -0.27  0.49 -0.76 -0.41  0.88
 20090827 -0.02 -0.42 -0.80  0.18  0.97
 20090828 -1.46 -0.63  0.10  0.34  0.92
 20090831 -1.39  0.36 -0.16  0.38 -3.52
 20090901 -0.55  0.80  0.68 -0.68  0.09
 20090902 -0.97 -0.19 -0.28 -0.26  1.37
 20090903 -0.28 -0.48  0.31  0.36  1.77
 20090904 -0.51 -0.06 -0.21  1.08 -1.50
 20090909  3.15 -1.72 -0.54 -0.47  0.49
 20090910  1.12 -0.06  0.66  0.05  0.16
 20090911  1.05  0.55  0.48 -0.33 -1.90
 20090914 -1.03  0.48  0.41  0.34  0.89
 20090915 -0.96 -0.19 -0.08  0.21  1.06
 20090916  7.44  0.95  1.13 -1.12 -1.50
 20090917 -0.21  0.13  0.78  0.48  2.79
 20090918 -0.55 -0.18 -0.42 -0.53 -0.73
 20090921  0.73 -0.02  1.55  0.58  8.71
 20090922  3.00 -0.49 -0.11  1.34 -5.10
 20090923 -0.63  0.18  0.71  0.59 -0.31
 20090924  0.13  0.52  0.08  1.32  0.42
 20090925 -1.29  0.55 -0.43 -1.07  0.63
 20090928  0.58 -2.73 -0.06 -0.19 -0.69


In [55]:
data_grid = data_list.transpose(1,2,0)

In [56]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from regain import utils; reload(utils)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_)
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_);
from regain.admm import latent_time_graph_lasso_v3_; reload(latent_time_graph_lasso_v3_);

mdl = latent_time_graph_lasso_v3_.LatentTimeGraphLasso(
    bypass_transpose=False, assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
    max_iter=500, rho=1. / np.array(data_list).shape[1])

ltgl = GridSearchCV(mdl, dict(#tau=[0.25, 0.2, 0.1],
                             tau=np.linspace(0.01, 0.8, 20),
                             eta=[0], beta=[0],
                             #alpha=[0.05, 0.1, 1],
                             alpha=np.linspace(0.01, 0.2, 5)),
                    cv=ShuffleSplit(1, test_size=.5)).fit(data_grid)

In [58]:
ltgl.best_estimator_.precision_

array([[[ 0.2663739 , -0.        , -0.        , -0.        ,  0.        ],
        [-0.        ,  0.98324308, -0.        , -0.        ,  0.        ],
        [-0.        , -0.        ,  1.41211198, -0.        , -0.01293394],
        [-0.        , -0.        , -0.        ,  0.96868926, -0.        ],
        [ 0.        ,  0.        , -0.01293394, -0.        ,  0.17808471]],

       [[ 0.06496547, -0.        , -0.        , -0.09513111,  0.        ],
        [-0.        ,  0.58114476,  0.        ,  0.        ,  0.        ],
        [-0.        ,  0.        ,  0.49464878,  0.        , -0.        ],
        [-0.09513111,  0.        ,  0.        ,  0.53323127, -0.        ],
        [ 0.        ,  0.        , -0.        , -0.        ,  0.08861144]],

       [[ 0.27226079, -0.        , -0.        , -0.        ,  0.        ],
        [-0.        ,  1.10773215, -0.        , -0.        ,  0.        ],
        [-0.        , -0.        ,  0.72082331, -0.        ,  0.        ],
        [-0.        ,

In [ ]:
['AMZN', 'IBM', 'GOOG', 'MSFT','APOL']